In [17]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import datetime

_EEG_RAW_COLNAMES = {
    'RAW_TP9':'TP9', 
    'RAW_TP10':'TP10', 
    'RAW_AF7':'AF7', 
    'RAW_AF8':'AF8'
}
_EEG_MUSE_COLNAMES = [
    'Delta_TP9','Delta_TP10','Delta_AF7','Delta_AF8',
    'Theta_TP9', 'Theta_TP10', 'Theta_AF7', 'Theta_AF8',
    'Alpha_TP9', 'Alpha_TP10', 'Alpha_AF7', 'Alpha_AF8',
    'Beta_TP9', 'Beta_TP10', 'Beta_AF7', 'Beta_AF8',
    'Gamma_TP9', 'Gamma_TP10', 'Gamma_AF7', 'Gamma_AF8'
]
_PR

class Calibration:
    def __init__(self, root_dir:str, eeg_filename:str, gaze_filename:str, calibration_filenames):
        self.root_dir = root_dir
        self.eeg_raw, self.eeg_muse, self.eeg_blinks = self.read_eeg(os.path.join(root_dir, eeg_filename))
        self.gaze_raw = pd.read_csv(os.path.join(root_dir, gaze_filename))
        self.calibrations_raw = [(Path(f).stem, pd.read_csv(os.path.join(root_dir, f))) for f in calibration_filenames]
        display(self.eeg_raw)

    def plot_raws(self):
        fig, axes = plt.subplots(3, 1, figsize=(15,10), sharex=True)


    @staticmethod
    def timestamp_to_unix_seconds(x):
        date_format = datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
        unix_seconds = datetime.datetime.timestamp(date_format)
        return unix_seconds
    @staticmethod
    def timestamp_to_unix_milliseconds(x):
        date_format = datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
        unix_seconds = datetime.datetime.timestamp(date_format)
        unix_milliseconds = int(unix_seconds * 1000)
        return unix_milliseconds
    @classmethod
    def read_eeg(cls, filepath:str):
        # Assertion to check file existence
        assert os.path.exists(filepath), f'EEG file {filepath} does not exist...'
        # Initial read
        df = pd.read_csv(filepath)
        # Generate timestamp features
        df['unix_ms'] = df['TimeStamp'].apply(cls.timestamp_to_unix_milliseconds)
        df['unix_sec'] = df['TimeStamp'].apply(cls.timestamp_to_unix_seconds)    
        df['rel_sec'] = df['unix_sec'] - df['unix_sec'].iloc[0]
        df['rel_ms'] = df['unix_ms'] - df['unix_ms'].iloc[0] 
        # separate blinks from raw data
        signals = df[df['Elements'].isna()]
        blinks = df[df['Elements']=='/muse/elements/blink']
        blinks = blinks[['TimeStamp', 'unix_sec', 'unix_ms', 'rel_sec', 'rel_ms']]    
        # Get raw data, then rename the features
        raw_df = signals[['unix_sec','unix_ms', 'rel_sec', 'rel_ms', *list(_EEG_RAW_COLNAMES.keys())]]
        raw_df.rename(columns=_EEG_RAW_COLNAMES, inplace=True)
        # Get processed data
        processed_df = signals[['unix_sec','unix_ms', 'rel_sec', 'rel_ms', *_EEG_MUSE_COLNAMES]]
        # return 
        return raw_df, processed_df, blinks

NameError: name '_PR' is not defined

In [18]:
_ROOT_DIR = os.path.join('.','samples','blink_calibration','Android_1')
_CAL_FILES = [
    '0.25-72.csv', '0.25-90.csv',
    '0.5-72.csv', '0.5-90.csv',
    '0.75-72.csv', '0.75-90.csv'
]
calibration_session = Calibration(_ROOT_DIR, 'eeg.csv', 'left_eye_gaze.csv', _CAL_FILES)

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_26108\26580775.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df.rename(columns=_EEG_RAW_COLNAMES, inplace=True)


,unix_sec,unix_ms,rel_sec,rel_ms,TP9,TP10,AF7,AF8
1,1.766036e+09,1766036381714,0.015,15,842.52747,828.82780,791.35530,799.01100
2,1.766036e+09,1766036381715,0.016,16,819.15753,822.38100,792.56410,794.17584
3,1.766036e+09,1766036381715,0.016,16,799.01100,823.99270,796.99634,795.38464
4,1.766036e+09,1766036381716,0.017,17,817.94870,821.57510,799.01100,802.23444
5,1.766036e+09,1766036381716,0.017,17,850.18317,824.39560,803.44324,805.05493
...,...,...,...,...,...,...,...,...
22467,1.766036e+09,1766036469313,87.614,87614,840.51280,860.65936,0.00000,1108.46150
22468,1.766036e+09,1766036469313,87.614,87614,841.31866,868.31500,0.00000,1119.74350
22469,1.766036e+09,1766036469313,87.614,87614,843.73627,867.10620,0.00000,1087.10620
22470,1.766036e+09,1766036469314,87.615,87615,850.58606,860.65936,0.00000,1036.33700
